In [ ]:
from pathlib import Path

base_dir = Path("../results/input/giri/THA/trade")

In [ ]:
! ls -lah {base_dir}

In [ ]:
import pandas

In [ ]:
trade_adm0 = pandas.read_csv(base_dir / "baci_sector_trade_THA.csv")
trade_adm0["GID_0"] = "THA"

In [ ]:
trade_adm0.groupby("sector").count()

In [ ]:
adm0_codes = sorted(
    list(set(trade_adm0.export_country_code) | set(trade_adm0.import_country_code))
)

In [ ]:
gva_adm1_all = pandas.read_csv(base_dir / "DOSE_V2.csv")
gva_adm1 = gva_adm1_all.query(f"year == 2018 and GID_0 in {adm0_codes}")[
    [
        "GID_0",
        "GID_1",
        "year",
        "pop",
        "ag_grp_pc_usd",
        "man_grp_pc_usd",
        "serv_grp_pc_usd",
    ]
].dropna()

for sector in ("ag", "man", "serv"):
    gva_adm1[sector] = gva_adm1["pop"] * gva_adm1[f"{sector}_grp_pc_usd"]

In [ ]:
(
    len(set(adm0_codes) - set(gva_adm1.GID_0.unique())),
    len(gva_adm1.GID_0.unique()),
    len(adm0_codes),
)

In [ ]:
gva_adm1_all[["GID_0", "country", "year"]].groupby("GID_0").max()

In [ ]:
wenz_codes = set(gva_adm1.GID_0.unique())
adm_codes_df = pandas.DataFrame({"GID_0": adm0_codes})
adm_codes_df["ADM1_available_in_wenz"] = adm_codes_df.GID_0.isin(wenz_codes)
adm_codes_df.to_csv(
    base_dir / "gdp_adm1_availability_for_THA_trade_partners.csv", index=False
)

In [ ]:
gva_adm1.head(2)

In [ ]:
gva_adm1_sector_tha = gva_adm1.drop(
    columns=[
        "GID_0",
        "year",
        "pop",
        "ag_grp_pc_usd",
        "man_grp_pc_usd",
        "serv_grp_pc_usd",
    ]
).set_index("GID_1")

# TODO
# population-weighted (even gdp pc) for gap-filling
# downscale partner O/Ds to ADM1
# find weighted centroid to use as representative node
# global ADM1 land routing grid
# TODO

gva_proportion_adm1_sector = (
    (gva_adm1_sector_tha / gva_adm1_sector_tha.sum())
    .rename(columns={"serv": "ser"})
    .reset_index()
    .melt(id_vars="GID_1", var_name="sector", value_name="proportion")
)
gva_proportion_adm1_sector["GID_0"] = "THA"
gva_proportion_adm1_sector.shape

In [ ]:
trade_adm1 = trade_adm0.merge(
    gva_proportion_adm1_sector,
    left_on=("GID_0", "sector"),
    right_on=("GID_0", "sector"),
    how="outer",
)
trade_adm1.trade_value_thousandUSD *= trade_adm1.proportion
trade_adm1.trade_quantity_tons *= trade_adm1.proportion
trade_adm1 = trade_adm1.drop(columns=["GID_0", "proportion"])

imports_adm1 = trade_adm1.query('import_country_code == "THA"').copy()
imports_adm1["export_zone"] = imports_adm1.export_country_code
imports_adm1["import_zone"] = imports_adm1.GID_1
imports_adm1["partner_GID_0"] = imports_adm1.export_country_code

exports_adm1 = trade_adm1.query('export_country_code == "THA"').copy()
exports_adm1["export_zone"] = exports_adm1.GID_1
exports_adm1["import_zone"] = exports_adm1.import_country_code
exports_adm1["partner_GID_0"] = exports_adm1.import_country_code

trade_adm1 = pandas.concat([exports_adm1, imports_adm1]).rename(
    columns={"GID_1": "THA_GID_1"}
)[
    [
        "export_country_code",
        "export_zone",
        "import_country_code",
        "import_zone",
        "THA_GID_1",
        "partner_GID_0",
        "sector",
        "trade_value_thousandUSD",
        "trade_quantity_tons",
    ]
]

In [ ]:
trade_adm1.to_csv("baci_sector_adm1_trade_THA.csv", index=False)
trade_adm1.head(2)

In [ ]:
trade_adm1_total = (
    trade_adm1.drop(
        columns=[
            "export_country_code",
            "export_zone",
            "import_country_code",
            "import_zone",
            "sector",
        ]
    )
    .groupby(["THA_GID_1", "partner_GID_0"])
    .sum()
)
trade_adm1_total.to_csv("baci_total_adm1_trade_THA.csv")
trade_adm1_total.head(2)